In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://bb6818473482:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 00:37:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from arantxa.spark_utils import SparkUtils
json_schema = SparkUtils.generate_schema([("id", "int"), ("json_col", "string")])
json_data = [
    (1, '{"name": "Alice", "age": 25, "payments": [34, 433, 54], "address": {"city": "New York", "zip": "10001"}}'),
    (2, '{"name": "Bob", "age": 30, "address": {"city": "Los Angeles", "zip": "90001"}}'),
    (3, '{"name": "Charlie", "age": 35, "address": {"city": "Chicago", "zip": "60601"}}')
]

df_json = spark.createDataFrame(json_data, json_schema)
df_json.show(truncate=False)

+---+--------------------------------------------------------------------------------------------------------+
|id |json_col                                                                                                |
+---+--------------------------------------------------------------------------------------------------------+
|1  |{"name": "Alice", "age": 25, "payments": [34, 433, 54], "address": {"city": "New York", "zip": "10001"}}|
|2  |{"name": "Bob", "age": 30, "address": {"city": "Los Angeles", "zip": "90001"}}                          |
|3  |{"name": "Charlie", "age": 35, "address": {"city": "Chicago", "zip": "60601"}}                          |
+---+--------------------------------------------------------------------------------------------------------+



In [5]:
from pyspark.sql.functions import get_json_object
df_json = df_json.withColumn("name", get_json_object(df_json.json_col, "$.name"))
df_json.show()

+---+--------------------+-------+
| id|            json_col|   name|
+---+--------------------+-------+
|  1|{"name": "Alice",...|  Alice|
|  2|{"name": "Bob", "...|    Bob|
|  3|{"name": "Charlie...|Charlie|
+---+--------------------+-------+



In [6]:
df_json = df_json.withColumn("age", get_json_object(df_json.json_col, "$.age"))
df_json.show()

+---+--------------------+-------+---+
| id|            json_col|   name|age|
+---+--------------------+-------+---+
|  1|{"name": "Alice",...|  Alice| 25|
|  2|{"name": "Bob", "...|    Bob| 30|
|  3|{"name": "Charlie...|Charlie| 35|
+---+--------------------+-------+---+



In [10]:
from pyspark.sql.functions import from_json
# Deine the schema of the JSON object
people_schema = SparkUtils.generate_schema([("name", "string"),
                                            ("age", "int"),
                                            ("payments", "array_int"),
                                            ("address", "struct")])
df_parsed = df_json.withColumn("parsed", from_json(df_json.json_col, people_schema))
df_parsed.printSchema()
df_parsed.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- json_col: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |    |-- payments: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- address: struct (nullable = true)

+---+--------------------------------------------------------------------------------------------------------+------------------------------+
|id |json_col                                                                                                |parsed                        |
+---+--------------------------------------------------------------------------------------------------------+------------------------------+
|1  |{"name": "Alice", "age": 25, "payments": [34, 433, 54], "address": {"city": "New York", "zip": "10001"}}|{Alice, 25, [34, 433, 54], {}}|
|2  |{"name": "Bob", "age": 30, "address": {"city": "Los Angeles", "zip": "90001"}}      

In [11]:
df_parsed.printSchema()

root
 |-- id: integer (nullable = true)
 |-- json_col: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: integer (nullable = true)
 |    |-- payments: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- address: struct (nullable = true)



In [12]:
from pyspark.sql.functions import col
df_parsed.select(col("parsed.name"), col("parsed.payments").getItem(1)).show()

+-------+------------------+
|   name|parsed.payments[1]|
+-------+------------------+
|  Alice|               433|
|    Bob|              NULL|
|Charlie|              NULL|
+-------+------------------+



In [14]:
df_a = spark.createDataFrame([(1, "Alice"), (2, "Bob")],
                              ["id", "name"])
df_a.show()
df_b = spark.createDataFrame([(3, "Charlie"), (4, "David")],
                              ["id", "name"])
df_b.show()
result = df_a.union(df_b)
result.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+

+---+-------+
| id|   name|
+---+-------+
|  3|Charlie|
|  4|  David|
+---+-------+

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  4|  David|
+---+-------+



In [15]:
sc.stop()